In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from statistics import mean

In [3]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2200.162
BogoMIPS:            4400.32
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_sin

In [4]:
from numpy import genfromtxt

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
X_train=genfromtxt("/content/drive/My Drive/9no Semestre/AEM_9sem/features/train_pca.csv", delimiter=',')
X_test=genfromtxt("/content/drive/My Drive/9no Semestre/AEM_9sem/features/test_pca.csv", delimiter=',')
y_train=genfromtxt("/content/drive/My Drive/9no Semestre/AEM_9sem/features/train_labels.csv", delimiter=',')
y_test=genfromtxt("/content/drive/My Drive/9no Semestre/AEM_9sem/features/test_labels.csv", delimiter=',')

In [7]:
len(X_train)

6829

In [8]:
len(X_test)

759

In [9]:
X_train = np.vstack((X_train, X_test))

In [10]:
len(X_train)

7588

In [11]:
X_train.shape

(7588, 10)

In [12]:
y_train.shape

(6829,)

In [13]:
y_test.shape

(759,)

In [14]:
y_train = np.hstack((y_train, y_test))

In [15]:
y_train.shape

(7588,)

In [16]:
Models=['SGDClassifier','SVMs','Random forest classifier','AdaBoostClassifier','KNeighborsClassifier']
TrainingTime=[]
Allaccuracies=[]
ValTime=[]
TotalTIme=[]

In [17]:
from time import time

#Test 0
*SGDClassifier*

In [18]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=10, shuffle=True, random_state=0)

In [19]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(random_state=0)
#clf.fit(X_train, y_train)

In [20]:
from sklearn.metrics import classification_report

In [21]:
def kfolds_applied(model):
  global kf
  global X_train
  global y_train
  model_ypred = []
  ttime=[]
  vtime=[]
  for train_index, val_index in kf.split(X_train):
    X_entrenamiento, X_val = X_train[train_index], X_train[val_index]
    y_entrenamiento, y_val = y_train[train_index], y_train[val_index]
 
    t0 = time()
    model.fit(X_entrenamiento,y_entrenamiento)
    ttime.append(time() - t0)

    t0 = time()
    y_pred = model.predict(X_val)
    vtime.append(time() - t0)
    
    model_ypred.append([y_pred,y_val])
  
  return model_ypred,ttime,vtime

In [22]:
model_ypred,ttime,vtim = kfolds_applied(clf)

In [23]:
print(mean(ttime))
print(mean(vtim))

0.027313590049743652
0.00039975643157958985


In [24]:
print(mean(ttime)+mean(vtim))

0.02771334648132324


In [26]:
def metrics(preds):
  accuracies_one=[]
  f1score0 = []
  precision0 = []
  recall0 = []
  f1score1 = []
  precision1 = []
  recall1 = []
  Dic0 = {}
  Dic1 = {}
  for i in preds:
    reporte = classification_report(i[0], i[1],output_dict=True)
    accuracies_one.append(reporte["accuracy"])
    f1score0.append(reporte["0.0"]['f1-score'])
    precision0.append(reporte["0.0"]['precision'])
    recall0.append(reporte["0.0"]['recall'])
    f1score1.append(reporte["1.0"]['f1-score'])
    precision1.append(reporte["1.0"]['precision'])
    recall1.append(reporte["1.0"]['recall'])
  Dic0['f1-score'] = f1score0
  Dic0['precision'] = precision0
  Dic0['recall'] = recall0
  Dic1['f1-score'] = f1score1
  Dic1['precision'] = precision1
  Dic1['recall'] = recall1
  return accuracies_one,Dic0,Dic1

In [27]:
accuracies_one,Dic0,Dic1 = metrics(model_ypred)

In [28]:
mean(accuracies_one)

0.8313153329787493

In [29]:
print(mean(Dic0['f1-score']))
print(mean(Dic0['precision']))
print(mean(Dic0['recall']))

0.8344564842605606
0.8513229197172266
0.8190489187478411


In [30]:
print(mean(Dic1['f1-score']))
print(mean(Dic1['precision']))
print(mean(Dic1['recall']))

0.8277762101153819
0.812276967685712
0.8447995745584749


#Support vector machines (SVMs)

In [31]:
from sklearn import svm
clf_SVC = svm.SVC(random_state=0)

In [32]:
model_ypred,ttime,vtim = kfolds_applied(clf_SVC)

In [33]:
print(mean(ttime))
print(mean(vtim))
print(mean(ttime)+mean(vtim))

1.1778918504714966
0.12197470664978027
1.2998665571212769


In [34]:
accuracies_one,Dic0,Dic1 = metrics(model_ypred)

In [35]:
mean(accuracies_one)

0.8819186472966443

In [36]:
print(mean(Dic0['f1-score']))
print(mean(Dic0['precision']))
print(mean(Dic0['recall']))

0.8832168511478181
0.8950056018596192
0.8718648299751055


In [37]:
print(mean(Dic1['f1-score']))
print(mean(Dic1['precision']))
print(mean(Dic1['recall']))

0.8801411345229632
0.8685529756325054
0.8921926952646302


#Random forest classifier

In [38]:
from sklearn.ensemble import RandomForestClassifier
clf_RF = RandomForestClassifier(random_state=0)

In [39]:
model_ypred,ttime,vtim = kfolds_applied(clf_RF)

In [40]:
print(mean(ttime))
print(mean(vtim))
print(mean(ttime)+mean(vtim))

1.9054789066314697
0.026489686965942384
1.931968593597412


In [41]:
accuracies_one,Dic0,Dic1 = metrics(model_ypred)

In [42]:
mean(accuracies_one)

0.8688727356158812

In [43]:
print(mean(Dic0['f1-score']))
print(mean(Dic0['precision']))
print(mean(Dic0['recall']))

0.8703424475539903
0.8801235909534949
0.8609668166585334


In [44]:
print(mean(Dic1['f1-score']))
print(mean(Dic1['precision']))
print(mean(Dic1['recall']))

0.8669713620199795
0.857646794978599
0.8767225717405864


# AdaBoostClassifier

In [45]:
from sklearn.ensemble import AdaBoostClassifier
clf_AdB = AdaBoostClassifier(n_estimators=100, random_state=0)

In [46]:
model_ypred,ttime,vtim = kfolds_applied(clf_AdB)

In [47]:
print(mean(ttime))
print(mean(vtim))
print(mean(ttime)+mean(vtim))

1.4236802339553833
0.02427396774291992
1.4479542016983034


In [48]:
accuracies_one,Dic0,Dic1 = metrics(model_ypred)

In [49]:
mean(accuracies_one)

0.8363210515154992

In [50]:
print(mean(Dic0['f1-score']))
print(mean(Dic0['precision']))
print(mean(Dic0['recall']))

0.8361539615443071
0.8372351733707201
0.8357628683585971


In [51]:
print(mean(Dic1['f1-score']))
print(mean(Dic1['precision']))
print(mean(Dic1['recall']))

0.8361259169090807
0.8360355383531013
0.8369574518165371


#KNeighborsClassifier

In [52]:
 from sklearn.neighbors import KNeighborsClassifier
 clf_neigh = KNeighborsClassifier()

In [53]:
model_ypred,ttime,vtim = kfolds_applied(clf_neigh)

In [54]:
print(mean(ttime))
print(mean(vtim))
print(mean(ttime)+mean(vtim))

0.009689927101135254
0.07581989765167237
0.08550982475280762


In [55]:
accuracies_one,Dic0,Dic1 = metrics(model_ypred)

In [56]:
mean(accuracies_one)

0.8667657416194757

In [57]:
print(mean(Dic0['f1-score']))
print(mean(Dic0['precision']))
print(mean(Dic0['recall']))

0.8698005975545178
0.8916612037712895
0.8491373578105139


In [58]:
print(mean(Dic1['f1-score']))
print(mean(Dic1['precision']))
print(mean(Dic1['recall']))

0.8631527694994102
0.8417884653872594
0.8858204498962187


In [59]:
# ResultsTime = pd.DataFrame(list(zip(Models, TrainingTime,ClassificationTime)),
#                columns =['Models', 'TrainingTime','ClassificationTime'])

In [60]:
# ResultsTime.to_csv("ResultsTimeAllDim.csv")